In [1]:
import pandas as pd
import numpy as np
import requests
import time

In [2]:
symbol = ['msft', 'aapl', 'amzn', 'adbe', 'qcom', 'orcl', 'acn', 'ibm', 'xom', 'intu',
         'pg', 'blk', 'ups', 'hd', 'mcd', 't', 'a', 'wmt', 'dis', 'tgt',
         'adi', 'jnj', 'mrk', 'mdt', 'syk', 'cl', 'el', 'ko', 'mmm', 'nke',
         'utx', 'lmt', 'axp', 'spg', 'mmc', 'all', 'k', 'low', 'gild', 'yum',
         'pld', 'stt', 'ip', 'nee', 'shw', 'ea', 'abt', 'unh', 'vrsn', 'lly']

In [3]:
for k in range(len(symbol)):
    r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol[k]}&outputsize=full&apikey=SVRXBBY37Z6W0BJZ").json()
    
    df = pd.DataFrame(r['Time Series (Daily)'])
    df = df.T
    
    date = []
    for row in df.index:
        date.append(row)
        
    df['date']=date
    df.set_index(np.arange(0,len(df)), inplace = True)
    df = df.iloc[31:4562]
    df.set_index(np.arange(0,len(df)), inplace = True)
    sym = r['Meta Data']['2. Symbol']
    symb = [sym]*len(df)
    df['symbol'] = symb
    
    year = []
    month = []
    quarter = []
    popen = []
    phigh = []
    plow = []
    pclose = []
    pvolume = []
    for i in range(len(df)):
        year.append(df.iloc[i]['date'].split("-")[0])
        month.append(df.iloc[i]['date'].split("-")[1])
        if month[i] == '12' or month[i] == '11' or month[i] == '10':
            quarter.append(4)
        elif month[i] == '09'or month[i] == '08' or month[i] == '07':
            quarter.append(3)
        elif month[i] == '06' or month[i] == '05' or month[i] == '04':
            quarter.append(2)
        else:
            quarter.append(1)
        popen.append(float(df.iloc[i]['1. open']))
        phigh.append(float(df.iloc[i]['2. high']))
        plow.append(float(df.iloc[i]['3. low']))
        pclose.append(float(df.iloc[i]['4. close']))
        pvolume.append(float(df.iloc[i]['5. volume']))
        
        
    df['year'] = year
    df['quarter'] = quarter
    
    del df['1. open']
    del df['2. high']
    del df['3. low']
    del df['4. close']
    del df['5. volume']
    
    df['open'] = popen
    df['high'] = phigh
    df['low'] = plow
    df['close'] = pclose
    df['volume'] = pvolume
    
    uu = np.where(df['close'] - df['open'] > 0, 1, 0)
    df["pup"] = uu
    
    vv = []
    for i in range(len(df)-1):
        vv.append(df.iloc[i]['volume'] - df.iloc[i+1]['volume'])
    
    vv.append(1)
    
    ww = []
    for j in range(len(vv)):
        if vv[j] > 0:
            vv[j] = 1
        else:
            vv[j] = 0
        if uu[j] == 1 and vv[j] == 1:
            ww.append(1)
        else:
            ww.append(0)
            
    df["vup"] = vv
    df["bothup"] = ww
    
    
    df.to_csv(f'raw/{symbol[k]}_raw.csv', index=False)
    time.sleep(12)